**[Geospatial Analysis Home Page](https://www.kaggle.com/learn/geospatial-analysis)**

---


# Introduction

You are a Starbucks big data analyst ([that’s a real job!](https://www.forbes.com/sites/bernardmarr/2018/05/28/starbucks-using-big-data-analytics-and-artificial-intelligence-to-boost-performance/#130c7d765cdc)) looking to find the next store into a [Starbucks Reserve Roastery](https://www.businessinsider.com/starbucks-reserve-roastery-compared-regular-starbucks-2018-12#also-on-the-first-floor-was-the-main-coffee-bar-five-hourglass-like-units-hold-the-freshly-roasted-coffee-beans-that-are-used-in-each-order-the-selection-rotates-seasonally-5).  These roasteries are much larger than a typical Starbucks store and have several additional features, including various food and wine options, along with upscale lounge areas.  You'll investigate the demographics of various counties in the state of California, to determine potentially suitable locations.

<center>
<img src="https://i.imgur.com/BIyE6kR.png" width="450"><br/><br/>
</center>

Before you get started, run the code cell below to set everything up.

In [ ]:
import math
import pandas as pd
import geopandas as gpd
from learntools.geospatial.tools import geocode

import folium 
from folium import Choropleth, Marker
from folium.plugins import HeatMap, MarkerCluster

from shapely.geometry import MultiPolygon

You'll use the `embed_map()` function from the previous exercise to visualize your maps.

In [ ]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

# Exercises

### 1) Geocode the missing locations.

Run the next code cell to create a DataFrame `starbucks` containing Starbucks locations in the state of California.

In [ ]:
# Load and preview Starbucks locations in California
starbucks = pd.read_csv("../input/geospatial-learn-course-data/starbucks_locations.csv")
starbucks.head()

Most of the stores have known (latitude, longitude) locations.  But, all of the locations in the city of Berkeley are missing.

In [ ]:
# How many rows in each column have missing values?
print(starbucks.isnull().sum())

# View rows with missing locations
rows_with_missing = starbucks[starbucks["City"]=="Berkeley"]
rows_with_missing

Use the code cell below to fill in these values with the OpenStreetMap Nominatim geocoder.

Note that in the tutorial, we used `geocode()` (from `geopandas.tools`) to geocode values, and this is what you can use in your own projects outside of this micro-course.  

In this exercise, you will use a slightly different function `geocode()` (from `learntools.geospatial.tools`).  This function was imported at the top of the notebook and works identically to the function from GeoPandas.

So, in other words, as long as: 
- you don't change the import statements at the top of the notebook, and 
- you call the geocoding function as `geocode()` in the code cell below, 

your code will work as intended!

In [ ]:
# Your code here
def my_geocoder(row):
    point = geocode(row, provider='nominatim').geometry[0]
    return pd.Series({'Longitude': point.x, 'Latitude': point.y})

berkeley_locations = rows_with_missing.apply(lambda x: my_geocoder(x['Address']), axis=1)
starbucks.update(berkeley_locations)

### 2) View Berkeley locations.

Let's take a look at the locations you just found.  Visualize the (latitude, longitude) locations in Berkeley in the OpenStreetMap style. 

In [ ]:
# Create a base map
m_2 = folium.Map(location=[37.88,-122.26], zoom_start=13)

# Add a marker for each Berkeley location
for idx, row in starbucks[starbucks["City"]=='Berkeley'].iterrows():
    Marker([row['Latitude'], row['Longitude']]).add_to(m_2)

# Show the map
embed_map(m_2, 'q_2.html')

### 3) Consolidate your data.

Run the code below to load a GeoDataFrame `CA_counties` containing the name, area (in square kilometers), and a unique id (in the "GEOID" column) for each county in the state of California.  The "geometry" column contains a polygon with county boundaries.

In [ ]:
CA_counties = gpd.read_file("../input/geospatial-learn-course-data/CA_county_boundaries/CA_county_boundaries/CA_county_boundaries.shp")
CA_counties.head()

Next, we create three DataFrames:
- `CA_pop` contains an estimate of the population of each county.
- `CA_high_earners` contains the number of households with an income of at least $150,000 per year.
- `CA_median_age` contains the median age for each county.

In [ ]:
CA_pop = pd.read_csv("../input/geospatial-learn-course-data/CA_county_population.csv", index_col="GEOID")
CA_high_earners = pd.read_csv("../input/geospatial-learn-course-data/CA_county_high_earners.csv", index_col="GEOID")
CA_median_age = pd.read_csv("../input/geospatial-learn-course-data/CA_county_median_age.csv", index_col="GEOID")

Use the next code cell to join the `CA_counties` GeoDataFrame with `CA_pop`, `CA_high_earners`, and `CA_median_age`.

Name the resultant GeoDataFrame `CA_stats`, and make sure it has 8 columns: "GEOID", "name", "area_sqkm", "geometry", "population", "high_earners", and "median_age".  Also, make sure the CRS is set to `{'init': 'epsg:4326'}`.

In [ ]:
# Joining all the following dataset's solumn into "cols_to_add"
cols_to_add = CA_pop.join([CA_high_earners, CA_median_age]).reset_index()
# Merging "cols_to_add" and "CA_stats" on GEOID
CA_stats = CA_counties.merge(cols_to_add, on="GEOID")

Now that we have all of the data in one place, it's much easier to calculate statistics that use a combination of columns.  Run the next code cell to create a "density" column with the population density.

In [ ]:
CA_stats["density"] = CA_stats["population"] / CA_stats["area_sqkm"]

### 4) Which counties look promising?

Collapsing all of the information into a single GeoDataFrame also makes it much easier to select counties that meet specific criteria.

Use the next code cell to create a GeoDataFrame `sel_counties` that contains a subset of the rows (and all of the columns) from the `CA_stats` GeoDataFrame.  In particular, you should select counties where:
- there are at least 100,000 households making \$150,000 per year,
- the median age is less than 38.5, and
- the density of inhabitants is at least 285 (per square kilometer).

Additionally, selected counties should satisfy at least one of the following criteria:
- there are at least 500,000 households making \$150,000 per year,
- the median age is less than 35.5, or
- the density of inhabitants is at least 1400 (per square kilometer).

In [ ]:
# Your code here
sel_counties = CA_stats[((CA_stats.high_earners > 100000) &
                         (CA_stats.median_age < 38.5) &
                         (CA_stats.density > 285) &
                         ((CA_stats.median_age < 35.5) |
                         (CA_stats.density > 1400) |
                         (CA_stats.high_earners > 500000)))]

### 5) How many stores did you identify?

When looking for the next Starbucks Reserve Roastery location, you'd like to consider all of the stores within the counties that you selected.  So, how many stores are within the selected counties?

To prepare to answer this question, run the next code cell to create a GeoDataFrame `starbucks_gdf` with all of the starbucks locations.

In [ ]:
starbucks_gdf = gpd.GeoDataFrame(starbucks, geometry=gpd.points_from_xy(starbucks.Longitude, starbucks.Latitude))
starbucks_gdf.crs = {'init': 'epsg:4326'}

So, how many stores are in the counties you selected?

In [ ]:
# Fill in your answer
locations_of_interest = gpd.sjoin(starbucks_gdf, sel_counties)
num_stores = len(locations_of_interest)
print(num_stores)

### 6) Visualize the store locations.

Create a map that shows the locations of the stores that you identified in the previous question.

In [ ]:
# Create a base map
m_6 = folium.Map(location=[37,-120], zoom_start=6)

# Show selected store locations in a decluster way using MarkerCluster
mc = MarkerCluster()

locations_of_interest = gpd.sjoin(starbucks_gdf, sel_counties)
for idx, row in locations_of_interest.iterrows():
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        mc.add_child(folium.Marker([row['Latitude'], row['Longitude']]))

m_6.add_child(mc)

# Show the map
embed_map(m_6, 'q_6.html')

### Looking where to build new Stores in San Francisco

In [ ]:
# Create "San_Francisco) df from "starbucks" 
San_Francisco = starbucks[starbucks["City"]=="San Francisco"]
# Creating a GeoDataFrame of "San_Francisco"
San_Francisco_gdf = gpd.GeoDataFrame(San_Francisco, geometry=gpd.points_from_xy(San_Francisco.Longitude, San_Francisco.Latitude))
# Adding the CRS data
San_Francisco_gdf.crs = {'init': 'epsg:4326'}
# Converting the ESPG to 2263 for unit purposes to convert in metres
San_Francisco_gdf = San_Francisco_gdf.to_crs(epsg=2263)

In [ ]:
# Creating a buffer of 1km radius to capture the coverage of each store in "San_Francisco"
buffer_in_meters = (1 * 1000) # Change the kms wished to cover
coverage = gpd.GeoDataFrame(geometry=San_Francisco_gdf.geometry).buffer(buffer_in_meters)

In [ ]:
# Create map with release incidents and monitoring stations
m_7 = folium.Map(location=[37.77,-122.4], zoom_start=12)
# View SF stores
for idx, row in San_Francisco_gdf.iterrows():
    Marker([row['Latitude'], row['Longitude']]).add_to(m_7)
# Plot each polygon on the map
folium.GeoJson(coverage.geometry.to_crs(epsg=4326)).add_to(m_7)
# Adding the Ltn and Lat data markers
folium.LatLngPopup().add_to(m_7)

embed_map(m_7, 'm_7.html')

In [ ]:
# Provide 2 new locations for potential new stores
# Proposed location of hospital 1
lat_1 = 37.7771
long_1 = -122.4650

# Proposed location of hospital 2
lat_2 = 37.7442
long_2 = -122.4136

# Do not modify the code below this line - Creating a new_df based on choosen location
new_df = pd.DataFrame(
        {'Latitude': [lat_1, lat_2],
         'Longitude': [long_1, long_2]})
new_gdf = gpd.GeoDataFrame(new_df, geometry=gpd.points_from_xy(new_df.Longitude, new_df.Latitude))
new_gdf.crs = {'init' :'epsg:4326'}
new_gdf = new_gdf.to_crs(epsg=2263) # Converting to ESPG 2263 to be on the same scale as previous map
# get new coverage
new_coverage = gpd.GeoDataFrame(geometry=new_gdf.geometry).buffer(buffer_in_meters)
# make the map
m_8 = folium.Map(location=[37.77,-122.4], zoom_start=12) 
folium.GeoJson(coverage.geometry.to_crs(epsg=4326)).add_to(m_8) # Adding existing coverage
folium.GeoJson(new_coverage.geometry.to_crs(epsg=4326)).add_to(m_8) # Adding new coverage
# Add new stores
for idx, row in new_gdf.iterrows():
    Marker([row['Latitude'], row['Longitude']], icon=folium.Icon(color="red")).add_to(m_8)
# View existing stores
for idx, row in San_Francisco_gdf.iterrows():
    Marker([row['Latitude'], row['Longitude']]).add_to(m_8)
folium.LatLngPopup().add_to(m_8)
# Display Map
display(embed_map(m_8, 'q_8.html'))